# Classifying Credit Score: Before the ML

Suppose I work for a company that gives out short-term personal loans. Before lending money to someone, I'd need some assurance that the loan would be repaid on time (or at least, an assurance I'd not lose all of my money—or, in some cases, the money of those who have invested in my company). In addition to finding out if a potential borrower can be trusted to make timely payments, I'd also need to find out how much money I can trust my borrower with (or how much I want to lend to a particular class of borrower) and at what interest rates.

One way to check if someone is reliable is to look at their credit history from a credit bureau like FICO. Lenders report information about their borrowers, such as loan amount, tenure, payment history, and interest rates, to credit bureaus at regular intervals, usually every 30 to 45 days. The credit bureau then gives each borrower a credit score, which gives me a fair idea of their reliability. But just looking at the score isn't enough. For example, paying off a loan on time might temporarily lower someone's score, which means the score itself is a partial indicator of reliability. In order to fully evaluate a potential borrower, I need more information.

Credit bureaus also record other information from lenders for different lengths of time. For example, Experian records open loans in good standing (i.e., loans where the borrower has been making timely repayments) for as long as they are active, while closed loans in good standing stay on the report for 10 years. Any missed / late payments or defaults are visible on the records for 7 years, and hard inquiries for 2 years. Combined with other information on the bureau and a potential borrower's loan application, I'd gain a fair idea of how they'd behave if my company provided them a loan.

However, this analysis comes in slightly further downstream. The first thing I'd like to know, would be who applied for a loan from my company, and what information I have about them.


# Question:
**Who applied for a loan at my company and what do I know about them?**


In [ ]:
#Defining the first quartile, Q1, the third quartile, Q3, and the inter-quartile range for 'Number of Credit Cards'
##

# First Quartile
Q1 = df['Interest Rate on Credit Cards'].quantile(0.25)

# Third Quartile
Q3 = df['Interest Rate on Credit Cards'].quantile(0.75)

# Inter - Quartile Range
IQR = Q3 - Q1


# Finding Q3 + 1.5*IQR
##
upper_bound = Q3 + 1.5 * IQR



# Grouping the number of credit cards in 'train' by customer ID, and aggregated by mode
##
mode_per_customer = train.groupby('Customer_ID')['Interest_Rate'].median()


# Defining a function to replacing the number of cards for those who fall above the defined upper bound with the mode
##
def replace_outliers(row):
    if row['Interest Rate on Credit Cards'] > upper_bound:
        # Find the mode for this specific 'Customer_ID' from 'train'
        return mode_per_customer.get(row['Customer ID'], row['Interest Rate on Credit Cards'])
    return row['Interest Rate on Credit Cards']

# Applying the function to the the dataframe
##
df['Interest Rate on Credit Cards'] = df.apply(replace_outliers, axis=1)

# Display the updated DataFrame
df


,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0
...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0


# Data
For the sake of this analysis, I have downloaded [this dataset](https://www.kaggle.com/datasets/parisrohan/credit-score-classification?select=train.csv) from Kaggle.


It contains two csv files, named 'train' and 'test', with the dependent variable being the column 'Credit_Score', which contains the values, 'Poor', 'Standard' and 'Good'.

Please note, this data is only a representation of the actual data used in analyses like this one. Actual data for credit score classification models requires a lot more cleaning than a dataset off of Kaggle, and contains a whole lot more variables.

# Importing Relevant Libraries

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

# Importing data

In [2]:
# Train
train = pd.read_csv('train.csv')
# Test
test = pd.read_csv('test.csv')

#Display Train Data
train

C:\Users\ANWITA\AppData\Local\Temp\ipykernel_30000\3460742305.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.866228,Poor
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.65161,Poor
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.809083,Poor
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.164979,Standard


In [3]:
# Display Test
test

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.84,3037.986667,2,...,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0x25fe5,CUS_0x8600,December,Sarah McBridec,4975,031-35-0942,Architect,20002.88,1929.906667,10,...,12.0,_,3571.7,34.780553,NaN,Yes,60.964772,146.48632477751087,Low_spent_Small_value_payments,275.53956951573343
49996,0x25fee,CUS_0x942c,September,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,7.0,Good,502.38,27.758522,31 Years and 11 Months,NM,35.104023,181.44299902757518,Low_spent_Small_value_payments,409.39456169535066
49997,0x25fef,CUS_0x942c,October,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,7.0,Good,502.38,36.858542,32 Years and 0 Months,No,35.104023,__10000__,Low_spent_Large_value_payments,349.7263321025098
49998,0x25ff0,CUS_0x942c,November,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,7.0,Good,502.38,39.139840,32 Years and 1 Months,No,35.104023,97.59857973344877,High_spent_Small_value_payments,463.23898098947717


In [4]:
# No. of Rows and Columns
##
print('train:', train.shape,  ## the first value in the tuple shows the no. of rows and the second, the no. of columns
      '\n',  ## '\n' works like the 'enter' key and shifts the subsequent values into the next line
      'test:', test.shape) ## note that 'test' contains one column less than 'train' in the output

train: (100000, 28) 
 test: (50000, 27)


*'train' contains 100,000 rows and 28 columns, while 'test' contains 50,000 rows and 27 columns*

*The columns of the two datasets are listed below*

In [5]:
# Columns
##
print('train:', train.columns,
      '\n', '\n',  ## '\n' has been used twice to increase the gap between the columns of 'train' and 'test' for better visibility
      'test:', test.columns) ## Note that the column 'Credit_Score' appears in 'train' but not in 'test'. All other columns are the same.

train: Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object') 
 
 test: Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_

In [6]:
# Months Covered by train and test
##
print(
    'Months covered by train:',
      train['Month'].unique(), ## Unique values in the sepecified column of a dataset
      '\n',
      '\n',   ## '\n' has been used twice to increase the gap between the columns of 'train' and 'test' for better visibility
     'Months covered by train:',
       test['Month'].unique()
     ) ## Note that 'train' covers months between January and August, and 'test' covers months between September and December

Months covered by train: ['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'] 
 
 Months covered by train: ['September' 'October' 'November' 'December']


***Note***

Note that 'train' and 'test' put together show the credit behaviour of a group of people from January to December of a given year - which makes them panel data (which is a mix of time series and cross section data, and tracks a set of observations over a period of time). Machine learning wouldn't work on the data in its current state, and I'd need to convert both 'train' and 'test' to cross section data before I try any machine learning on it.

Moreover, this exercise would provide the basis to answer the question above:

*Who's applying for a loan at my company, and what do I know about them?*


# Converting the data to cross section

## train

## Step 1 - Laying the foundations down: Building the actual dataset

In [7]:
## Creating a dictionary called 'data' for a single key called 'Customer Id'
  # The values associated with this key are the unique values in the column 'Customer_ID' of 'train'
  # The other columns will be added as we go along
##
data = {'Customer ID':train['Customer_ID'].unique()}

# Converting data into a pandas dataframe
##
df = pd.DataFrame(data)

# Displaying the dataframe
df

,Customer ID
0,CUS_0xd40
1,CUS_0x21b1
2,CUS_0x2dbc
3,CUS_0xb891
4,CUS_0x1cdb
...,...
12495,CUS_0x372c
12496,CUS_0xf16
12497,CUS_0xaf61
12498,CUS_0x8600


*This datset now have 12,500 rows and a single column. Which is to say, there are 12,500 unique Customer IDs, which have been converted to a single column dataset. Now, we add other columns to this dataset.*

## Step 2: Adding columns

### 1. Customer's Name

In [8]:
# Grouping customer's name by customer ID
# Each person has a unique customer ID, so using the first value in the column 'Name' (in train) for each 'Customer_ID'
##
name = train.groupby('Customer_ID')['Name'].first()


# Converting the above grouping into a dictionary
##
name_dict = name.to_dict()

# Mapping the values for 'Name' in the dictionary above to the column 'Customer ID' in the dataframe being created (df)
# The new column is called 'Name'
##
df['Name'] = df['Customer ID'].map(name_dict)

# Display the dataframe
df


,Customer ID,Name
0,CUS_0xd40,Aaron Maashoh
1,CUS_0x21b1,Rick Rothackerj
2,CUS_0x2dbc,Langep
3,CUS_0xb891,Jasond
4,CUS_0x1cdb,Deepaa
...,...,...
12495,CUS_0x372c,Lucia Mutikanik
12496,CUS_0xf16,Maria Sheahanb
12497,CUS_0xaf61,Chris Wickhamm
12498,CUS_0x8600,Sarah McBridec


### 2. Age and Age Group

I'll drop 'Age' as soon as I've built 'Age Group' - this would make it easier in later analysis

In [9]:
# Extracting the numeric parts of the age records in 'train'
##
train['Age_corrected'] = train['Age'].str.extract(r'(\d+)')

# Converting "Age" in "train" to integer for easier processing
##
train['Age_corrected'] = train['Age_corrected'].astype(int)

# Checking the values of Age
train['Age_corrected'].describe()

count    100000.000000
mean        119.509700
std         684.757313
min          14.000000
25%          25.000000
50%          34.000000
75%          42.000000
max        8698.000000
Name: Age_corrected, dtype: float64

*Note that the mean, standard deviation and maximum value far beyond the average human lifespan. Even the minimum of the column looks sketchy, since it's illegal to lend money to a minor (or even let them sign a financial contract) in many places. Lenders often set up a minimum age of 18 - 21 years old to qualify for a loan on their platform, based on their internal lending policy.*

*This signifies I'll need to do a sanity check on this data, and I'll start with the percentage of values in this column which are outliers. By outliers, I mean any value that's less than ($Q_{1} - 1.5 × IQR$), and greater than ($Q_{3} + 1.5 × IQR$), where $Q_{1}$ and $Q_{3}$ are the first and third quartiles respectively.*

In [10]:
# Calculate Q1 (25th percentile or 1st quartile) and Q3 (75th percentile or 3rd quartile)
##
Q1 = train['Age_corrected'].quantile(0.25)
Q3 = train['Age_corrected'].quantile(0.75)

# Calculate the Interquartile Range (IQR)
##
IQR = Q3 - Q1

# Define the lower and upper boundaries
##
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Find the number of values outside these boundaries
##
outliers = train[(train['Age_corrected'] < lower_bound) | (train['Age_corrected'] > upper_bound)]

# Calculate the percentage of outlier values
##
outlier_percentage = (len(outliers) / len(train)) * 100

# Display the percentage
print(f"Percentage of values outside the boundaries: {outlier_percentage:.2f}%")

Percentage of values outside the boundaries: 2.78%


*Since only 2.78% of the values in the column are outliers, it's highly likely that most people had correct records throughout, but some people had correct records for the most part, but a random abberration crept into the records for one or more months, which was large enough to drag the mean and standard deviation upwards. Luckily enough, 'train' contains data for eight months, and it's easy to simply take the most commonly occurring value of age (i.e. the mode) for each applicant, and see if it makes more sense. I'm working on the assumption that aberrations crept into some records for some people, but most of their age records over the months would still be correct.*

In [11]:
# Grouping 'Age' in 'train' by 'Customer_ID'
##
age = train.groupby('Customer_ID')['Age_corrected'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])

# Converting the groupings into a dictionary
##
age_dict = age.to_dict()

# Mapping the values for 'Age_corrected' in the dictionary above to the column 'Customer ID' in the dataframe being created (df)
# The new column is called 'Age'
##
df['Age'] = df['Customer ID'].map(age_dict)

# Display the dataframe
df

,Customer ID,Name,Age
0,CUS_0xd40,Aaron Maashoh,23
1,CUS_0x21b1,Rick Rothackerj,28
2,CUS_0x2dbc,Langep,34
3,CUS_0xb891,Jasond,55
4,CUS_0x1cdb,Deepaa,21
...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,18
12496,CUS_0xf16,Maria Sheahanb,45
12497,CUS_0xaf61,Chris Wickhamm,49
12498,CUS_0x8600,Sarah McBridec,28


In [ ]:
## Checking the values in the column 'Age'
df['Age'].describe()

count    12500.000000
mean        33.274560
std         10.764815
min         14.000000
25%         24.000000
50%         33.000000
75%         42.000000
max         56.000000
Name: Age, dtype: float64

*Note that the minimum age is still 14. Which means, I'll still need to find the number of people whose age is below 18 years old (which is the age of legal majority).*

 *In real life data, this could likely be an error in recording or retriving the data from the database. Or the applicant was legitimately less than 18 years old.*  

 *To solve the issue in real life, I'd first go back to the company's database to recheck the values of Age for them.If the correct ages are recorded in the database (and this is just an error that crept into the retrieval process), I could just replace the erroneous values with the correct ones and move on.*

*If it was an error in recording the data, I'd most likely need to get the applicant's date of birth from bureau records (or an identity document they've supplied), and find the correct age from there.*

*If an applicant is legitimately less than 18 years of age (or the minimum age requirement in the company's lending policy), I'd not give them a loan regardless of if they met the other elegibility criteria by whatever stroke of luck.*

*However, since none of the above options are available to me, I'll first check what percent of people are less than 18 years old (as per this dataset, at least), and go from there.*

In [13]:
# Separating those aged below 18
##
minors = df[(df['Age'] < 18) ]

# Calculate the percentage of outlier values
##
minor_percentage = (len(minors) / len(df)) * 100

# Display the percentage
##
print(f"Percentage of people younger than 18 years of age: {minor_percentage:.2f}%")


Percentage of people younger than 18 years of age: 5.90%


*Note that 5.9% of the 12,500 applicants in my data (which comes out to 737 people) have a recorded age of less than 18 years old.*

*Instead of wholesale dropping the affected appliants from the data (since they are still applicants, regardless of why their age is showing up as less than 18 years old - moreover, regulators often demand a certain minimum data be stored), I'll simply give them a category of their own when I build age groups and move on. This would mean I'm not losing data on purpose, while still marking the affected individuals for easier identification.*

**Creating Age Groups**

In [14]:
# Defining the edges of the bins to divede the data into.
##
bins = [ 0, 18, 25, 35, 45, 60]

# Defining the labels for each bin.
##
labels = ['Z- Incorrect Record / Minor', 'A- 18 - 24', 'B- 25 - 34', 'C- 35 - 44', 'D- 45 - 59']

# Using the bins and Labels defined above to form 'Age Group' from 'Age'
##
df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)

# Dropping 'Age from the dataframe
##
df = df.drop('Age', axis = 1)

# Displaying the dataframe
df

,Customer ID,Name,Age Group
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34
2,CUS_0x2dbc,Langep,B- 25 - 34
3,CUS_0xb891,Jasond,D- 45 - 59
4,CUS_0x1cdb,Deepaa,A- 18 - 24
...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34


*Peeping at the distribution of applicants by age*

In [15]:
# Finding the number of occurences of each age group

df['Age Group'].value_counts()

Age Group
B- 25 - 34                     3650
C- 35 - 44                     3581
A- 18 - 24                     2419
D- 45 - 59                     2113
Z- Incorrect Record / Minor     737
Name: count, dtype: int64

*Looks like a over half my applicants (7,231 people from 12,500 total applicants) are between 25 and 44 years old, while, in real life, I'd need to go back to the databse to verify the age records of 737 people.*

### 3. Occupation

In [16]:
# Grouping the "Occupation" in train by 'Customer_ID'.
## Assuming people stay in an occupation for a while, it's reasonable to fill in the first value if 'Occupation' for each customer ID
##
occupation = train.groupby('Customer_ID')['Occupation'].first()

# Converting the above grouping into a dictionary
##
occupation_dict = occupation.to_dict()

# # Mapping the values for 'Occupation' in the dictionary above to the column 'Customer ID' in the dataframe being created (df)
# The new column is called 'Occupation'
##
df['Occupation'] = df['Customer ID'].map(occupation_dict)

# Display the dataframe
df


,Customer ID,Name,Age Group,Occupation
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,_______
2,CUS_0x2dbc,Langep,B- 25 - 34,_______
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer
...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media_Manager
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect


In [17]:
# Checking the values in the column
df['Occupation'].unique()

array(['Scientist', '_______', 'Entrepreneur', 'Developer', 'Lawyer',
       'Media_Manager', 'Doctor', 'Journalist', 'Teacher', 'Engineer',
       'Manager', 'Accountant', 'Musician', 'Mechanic', 'Writer',
       'Architect'], dtype=object)

*Note the long dash among the unique values. Since there's no database I can go back and re-check these values, I'll replace this value with "Not Available". Plus, 'Media_Manager' can be replaced with 'Media Manager'.*

In [18]:
# Replacing the dash with 'Not Available' and 'Media_Manager' with 'Media Manager
##
df['Occupation'] = df['Occupation'].replace({'_______': 'Not Available','Media_Manager': 'Media Manager'})

# Dispaying the dataframe
df

,Customer ID,Name,Age Group,Occupation
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer
...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect


### 4. Count of Months for which data is available.

*Note, I'll use this column as a point of reference, or for calculating certain averages*

In [19]:
# Grouping the "Month" in train by 'Customer_ID' and aggregating by count
##
months = train.groupby('Customer_ID')['Month'].count()

# Converting the groupings into a dictionary
##
months_dict = months.to_dict()

# # Mapping the values for 'Month' in the dictionary above to the column 'Customer ID' in the dataframe being created (df)
# The new column is called 'Count of Months'
##
df['Count of Months'] = df['Customer ID'].map(months_dict)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8
...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8


### 5. Annual Income

In [20]:
# Grouping the "Annual_Income" in train by 'Customer_ID'
# Since there's one value of annual income in a single year, it makes no difference if the first value is mapped to customer ID
##
annual_income = train.groupby('Customer_ID')['Annual_Income'].first()

# Converting the mappings to a dictionary
##
annual_income_dict = annual_income.to_dict()

# Mapping the values for 'Annual_Income' in the dictionary above to the column 'Customer ID' in the dataframe being created (df)
# The new column is called 'Annual Income'
##
df['Annual Income'] = df['Customer ID'].map(annual_income_dict)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71_
...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.1
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88


In [21]:
# Checking the values in the column

df['Annual Income'].dtype

dtype('O')

*Since the column for 'Annual Income' consists of 'object' (non - numeric) type data, I'd need to convert the column to a numeric data type before moving forward.*

In [22]:
# Extracting the numeric parts of the values in the column & converting the column to 'float' data type.
##
df['Annual Income'] = df['Annual Income'].astype(str).str.replace(r'[^0-9.]', '', regex=True).astype(float)

# Display the dataset
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71
...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88


### 6. Average Monthly Income
*Since I have the annual income (which I can presume to be an aggregation for 12 months, or one full calendar / accounting year), it would make more sense to find the average monthly income by dividing the annual income by 12 instead of by the no. of months for which I have data on a person*

*Had I data for the total income over, say, the no. of months for which data is available, I would divide total income by the number of months to get more accurate results.*

*Moreover, considering a 12 month period offers me a degree of consistency in my data, especially when 'train' and 'test' put together cover a full calendar year.*

In [23]:
# Finding the average monthly income & rounding the result off to 2 decimals
##
df['Average Monthly Income'] = (df['Annual Income'] / 12).round(2)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31
...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91


### 7. Credit History in Years
*The credit history (or bureau vintage) shows how long someone has been borrowing for.*

In [24]:
# Extracting the numeric parts of the column labelled 'Credit_History_Age' & converting it into an integer.
##
train['Vintage in Years'] = train['Credit_History_Age'].str.extract(r'(\d+)')

# Filling the Null values with 0
train['Vintage in Years'] = train['Vintage in Years'].fillna(0)

# Converting the column to integer
train['Vintage in Years'] = train['Vintage in Years'].astype(int)

# Grouping the "Vintage in Years" in train by 'Customer_ID'
# Since 'train' and 'test' cover a single calendar year between them, I'll pick the first value of vintage for convenience.
##
vintage = train.groupby('Customer_ID')['Vintage in Years'].first()

# Converting the groupings into a dictionary
##
vintage_dict = vintage.to_dict()


# Mapping the values for 'Vintage in Years' in the dictionary above to the column 'Customer ID' in the dataframe being created (df)
# The new column is called 'Vintage in Years'
##

df['Vintage in Years'] = df['Customer ID'].map(vintage_dict)

# Display the dataset
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30
...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5


*Note that some people have apparently been on the bureau for longer than they've been alive, but there's very little I can do about this aberration here.*

*For real life data, I'd have to extract the data for the affected people again to check if it was an extraction error to begin with.*

*If it was a recording error, I'd probably need to calculate the number of years from their very first recorded loan / credit card (i.e. essentially make an educated guess that someone has been on the bureau for at least the length of time it's been since their first recorded loan / credit card).*

### 8. No. of Bank Acocunts

In [25]:
# Grouping the no. of bank accounts by customer_ID
# The average no. of bank accounts should even out any accounts that a person closed during the year, or any new ones they've opened
##
bank_accounts = train.groupby('Customer_ID')['Num_Bank_Accounts'].mean().round(0).astype(int)

# Converting the groupings to a dictionary
##
bank_accounts_dict = bank_accounts.to_dict()

# Mapping the dictionary onto the dataset
##
df['Number of Bank Accounts'] = df['Customer ID'].map(bank_accounts_dict)

# Display the Dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7
...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10


*Taking a look at the descriptive statistics for the column*

In [26]:
# Finding the Descriptive Statistics for the column
df['Number of Bank Accounts'].describe()

count    12500.000000
mean        17.092080
std         41.516251
min         -1.000000
25%          4.000000
50%          6.000000
75%          8.000000
max        411.000000
Name: Number of Bank Accounts, dtype: float64

*Note that the minimum number of bank accounts is $( - 1)$, and the maximum number of bank accounts is 411 - which ranges from highly unlikely to impossible. However, only 25% of the people on this dataset seem to have between 8 and 411 bank accounts.*

*To solve this, I'll replace those who have over $(Q_{3} + 1.5 \times IQR)$ accounts (where $Q_{3}$ is the $3^{rd}$ quartile, and $IQR = Q_{3} - Q_{1}$, where $Q_{1}$ is the first quartile) with the mode of the number of bank accounts.*

*As for those with $(-1)%$ bank accounts, the easiest way to deal with it would be to replace $(-1)$ with 0, while assuming that the affected people do not have a bank account (which would make some sense, as people cannot have a negative number of bank accounts, and the next best case, give our lack of further information, would be that they have no bank accounts). In real life, however, I'd need to go back to the company's database to check if the negative number was an error in recording the data, or an error in retrieving it.*

In [27]:
#Defining the first quartile, Q1, the third quartile, Q3, and the inter-quartile range for 'Number of Credit Cards'
##

# First Quartile
Q1 = df['Number of Bank Accounts'].quantile(0.25)

# Third Quartile
Q3 = df['Number of Bank Accounts'].quantile(0.75)

# Inter - Quartile Range
IQR = Q3 - Q1


# Finding Q3 + 1.5*IQR
##
upper_bound = Q3 + 1.5 * IQR



# Grouping the number of credit cards in 'train' by customer ID, and aggregated by mode
##
mode_per_customer = train.groupby('Customer_ID')['Num_Bank_Accounts'].agg(lambda x: x.mode()[0])


# Defining a function to replacing the number of cards for those who fall above the defined upper bound with the mode
##
def replace_outliers(row):
    if row['Number of Bank Accounts'] > upper_bound:
        # Find the mode for this specific 'Customer_ID' from 'train'
        return mode_per_customer.get(row['Customer ID'], row['Number of Bank Accounts'])
    return row['Number of Bank Accounts']

# Applying the function to the the dataframe
##
df['Number of Bank Accounts'] = df.apply(replace_outliers, axis=1)


# Display the updated DataFrame
df


,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7
...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10


In [28]:
# Handling the negative values
##
df['Number of Bank Accounts'] = df['Number of Bank Accounts'].replace({-1:0})

# Display the descriptive statistics of the column
##
df['Number of Bank Accounts'].describe()

count    12500.000000
mean         5.389280
std          2.607291
min          0.000000
25%          3.000000
50%          6.000000
75%          7.000000
max         14.000000
Name: Number of Bank Accounts, dtype: float64

*Now, the number of bank accounts ranges from 0 to 14. While 14 is a lot of bank accounts to have, it's a perfectly legal (and somewhat reasonable) number. Moreover, someone with 411 bank accounts would've raised a few eyebrows with the financial and legal authorities - first for the sheer amount of money and volume of transactions that would need that many accounts, and then for who are the people making and receiving payments from these bank accounts* 

### 9. No. of Credit Cards

In [29]:
# Grouping the no. of credit cards by customer_ID
# The average no. of credit cards should even out any cards that a person closed during the year, or any new ones they've taken
##
credit_cards = train.groupby('Customer_ID')['Num_Credit_Card'].mean().round(0).astype(int)

# Converting the groupings to a dictionary
##
credit_cards_dict = credit_cards.to_dict()

# Mapping the dictionary to the dataframe
##
df['Number of Credit Cards'] = df['Customer ID'].map(credit_cards_dict)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,177
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5
...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8


*Note that the person on the second row (the row with an index of 1) apparently has an average of 177 cards, which is unlikely for a single person. They probably have an extreme value in the corresponding column of the original dataset, which is likely dragging the mean upwards. Moreover, there are likely other people who have a similar problem.*

*To guage the severity of the problem, I'll first print the descriptive statistics of this column*

In [30]:
# Descriptive stats
##
df['Number of Credit Cards'].describe()

count    12500.000000
mean        22.476320
std         45.637599
min          0.000000
25%          4.000000
50%          6.000000
75%          8.000000
max        372.000000
Name: Number of Credit Cards, dtype: float64

*While 75% of the people have between 0 and 8 credit cards, the maximum no. of credit cards is 372, which means, 25% of the people in my dataset have between 8 and 372 cards. This group has probably driven the standard deviation up to 45.6 (~ 46) cards, and with it, the variance in the data (since the variance is simply the standard deviation squared).*

*The few large values in the data have also driven the mean up to 22.47 (~ 22) cards, even though only 25% people in this dataset have over 8 cards.*

*To solve this, I'll replace those who have over $(Q_{3} + 1.5 \times IQR)$ cards (where $Q_{3}$ is the $3^{rd}$ quartile, and $IQR = Q_{3} - Q_{1}$, where $Q_{1}$ is the first quartile) with the mode of the number of cards they own.*

*Note that the descriptive statistics table right above this text block states that $Q_{3}$ (or the value against the 75% mark) is 8, and $Q_{1}$ (the value against the 25% mark) is 4, which makes the IQR equal to 4. Subtracting $1.5 \times 4 = 6$ from $Q_{1}$ gives me $(-2)$ , and nobody can have a negative number of cards. Moreover, the minimum no. of cards is 0. This is why I have ignored the values less than ($Q_{1} - 1.5 \times IQR$).*


In [31]:
#Defining the first quartile, Q1, the third quartile, Q3, and the inter-quartile range for 'Number of Credit Cards'
##

# First Quartile
Q1 = df['Number of Credit Cards'].quantile(0.25)

# Third Quartile
Q3 = df['Number of Credit Cards'].quantile(0.75)

# Inter - Quartile Range
IQR = Q3 - Q1


# Finding Q3 + 1.5*IQR
##
upper_bound = Q3 + 1.5 * IQR



# Grouping the number of credit cards in 'train' by customer ID, and aggregated by mode
##
mode_per_customer = train.groupby('Customer_ID')['Num_Credit_Card'].agg(lambda x: x.mode()[0])


# Defining a function to replacing the number of cards for those who fall above the defined upper bound with the mode
##
def replace_outliers(row):
    if row['Number of Credit Cards'] > upper_bound:
        # Find the mode for this specific 'Customer_ID' from 'train'
        return mode_per_customer.get(row['Customer ID'], row['Number of Credit Cards'])
    return row['Number of Credit Cards']

# Applying the function to the the dataframe
##
df['Number of Credit Cards'] = df.apply(replace_outliers, axis=1)

# Display the updated DataFrame
df


,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5
...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8


*Now I'll take another look at the column.*

In [32]:
# Descriptive statistics

df['Number of Credit Cards'].describe()

count    12500.000000
mean         5.571600
std          2.114283
min          0.000000
25%          4.000000
50%          5.000000
75%          7.000000
max         14.000000
Name: Number of Credit Cards, dtype: float64

*On replacing the values, the column now shows that all people now have between 0 and 14 cards, which makes a lot more sense (though 14 is still a lot of credit cards, though not unreasonably so).*

*Note that the mean is now 5.57 (~ 6) cards, and the standard deviation is now 2.11 (~ 2) cards. This makes the variance 4.4521 (~ 4) cards.*

### 10. Interest Rate on Credit Card(s)

In [33]:
# Grouping Interest Rate by Customer ID and aggregating by mean
##
interest = train.groupby('Customer_ID')['Interest_Rate'].mean()

# Converting the grouping into a dictionary
##
interest_dict = interest.to_dict()

# Mapping this dictionary into the dataframe being created
##
df['Interest Rate on Credit Cards'] = df['Customer ID'].map(interest_dict)

# Display the dataset
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.00
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.00
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.00
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.00
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.00
...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.00
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.00
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.00
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.00


In [34]:
# Descriptive Statistics on the interest rate on credit cards

df['Interest Rate on Credit Cards'].describe()

count    12500.000000
mean        72.466040
std        165.067206
min          1.000000
25%          8.000000
50%         16.000000
75%         27.000000
max       1601.750000
Name: Interest Rate on Credit Cards, dtype: float64

*Note that the mean interest is $~ 72.47\%$, with a standard deviation of $~165.06\%$, both of which look predatory, and are far higher than the interest rate on the typical credit card. Moreover, $25\%$ of the people in this dataset are apparently paying between $~ 27\%$ and $~ 1601.75\%$ in credit card interest. While some values on the higher side may be a result of late or missed payments on credit card bills, the values that look positively predatory are more likely to be a result of aberrations in the data. To deal with this, I'll replace the values above $Q_{3} + 1.5 \times IQR$ with the median of interest rate grouped by Customer ID.*

In [35]:
#Defining the first quartile, Q1, the third quartile, Q3, and the inter-quartile range for 'Number of Credit Cards'
##

# First Quartile
Q1 = df['Interest Rate on Credit Cards'].quantile(0.25)

# Third Quartile
Q3 = df['Interest Rate on Credit Cards'].quantile(0.75)

# Inter - Quartile Range
IQR = Q3 - Q1


# Finding Q3 + 1.5*IQR
##
upper_bound = Q3 + 1.5 * IQR



# Grouping the number of credit cards in 'train' by customer ID, and aggregated by mode
##
mode_per_customer = train.groupby('Customer_ID')['Interest_Rate'].median()


# Defining a function to replacing the number of cards for those who fall above the defined upper bound with the mode
##
def replace_outliers(row):
    if row['Interest Rate on Credit Cards'] > upper_bound:
        # Find the mode for this specific 'Customer_ID' from 'train'
        return mode_per_customer.get(row['Customer ID'], row['Interest Rate on Credit Cards'])
    return row['Interest Rate on Credit Cards']

# Applying the function to the the dataframe
##
df['Interest Rate on Credit Cards'] = df.apply(replace_outliers, axis=1)

# Display the updated DataFrame
df


,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0
...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0


In [36]:
# Descriptive statistics for the column

df['Interest Rate on Credit Cards'].describe()

count    12500.000000
mean        14.732710
std          9.006463
min          1.000000
25%          8.000000
50%         13.000000
75%         20.000000
max         55.500000
Name: Interest Rate on Credit Cards, dtype: float64

*Note that the maximum interest rate is still $55.5\%$, which is still high - but they no longer look all that predatory.*

### 11. Number of Loans

*Counting the total number of loans a person has accumulated is not as simple as just adding the numbers up. Even some of the smaller personal loans have minimum tenures of 1 to 3 months, and the loan remains 'active' for the entirety of its tenure. In other words, when considering the number of active loans someone might have on their credit bureau records, the same loan will appear repeatedly each month until it reaches the end of its tenure. So if I simply add the number of loans up, I'll most likely count the same loan more than once, and end up with an artificially inflated number.*

*One way of finding an accurate number of loans that a person has taken in the time they've been on a credit bureau, is to look at the number of active loans in the most recent month, and add it to the number of loans that they have already paid off.To look at how many active loans someone generally has in a month, I'd probably want to look at the most commonly occurring figure (i.e. the mode).* 

*The data card for this dataset defines the column 'Num_of_Loan' in train as the number of loans taken from the bank, with no information on how many of these are currently active, or the time frame within which these loans were taken.* 

In [37]:
# Converting the 'Num_of_Loan' column in train to a numeric data type
##
train['Num_of_Loan'] = train['Num_of_Loan'].astype(str).str.replace(r'[^0-9]', '', regex=True).astype(int)

# Grouping the Number of Loans by customer ID and aggregating by mode
##
loans = train.groupby('Customer_ID')['Num_of_Loan'].agg(lambda x: x.mode()[0]).round(0).astype(int)

# Converting the groupings into a dictionary
##
loans_dict = loans.to_dict()

# Mapping the dictionary to the dataframe being created
##
df['Number of Loans per Month'] = df['Customer ID'].map(loans_dict)

# Display the dataframe
df
#

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards,Number of Loans per Month
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0,4
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0,1
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0,3
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0,1
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0,1
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0,4
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0,3
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0,5


In [38]:
df['Number of Loans per Month'].describe()

count    12500.000000
mean         3.532880
std          2.446442
min          0.000000
25%          2.000000
50%          3.000000
75%          5.000000
max          9.000000
Name: Number of Loans per Month, dtype: float64

*Which is to say, the most frequently occurring number of loans in a month ranges from 0 to 9, with a mean of 3.5 (approx. 4) and standard deviation of 2.4 (approx. 2).*

### 12. Most Common Loan Types
*The most common type of loans that a person has taken gives me an insight into their financial situation and / or repayment behaviour, especially when combined with how frequently they have taken these loans. For example, I may assume that someone who has taken multiple personal loans in the same month is in a tight financial situation, while someone who has one or more secured loans also has assets they have offered as collateral.*   

In [39]:
# Grouping type of loan by customer ID and aggregating by mode
##
most_common_loan_type = train.groupby('Customer_ID')['Type_of_Loan'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc)

# Converting the grouping into a dictionary
##
most_common_loan_type_dict = most_common_loan_type.to_dict()

# Mapping the dictionary onto the dataframe being created
##
df['Most Common Loan Type'] = df['Customer ID'].map(most_common_loan_type_dict)

# Ensuring those who have no loans are labelled as such
##
# Replace values in 'Most Common Loan Type' where 'Number of Loans per Month' is 0
df.loc[df['Number of Loans per Month'] == 0, 'Most Common Loan Type'] = 'No Loans'

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards,Number of Loans per Month,Most Common Loan Type
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,..."
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0,1,Credit-Builder Loan
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0,3,"Auto Loan, Auto Loan, and Not Specified"
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0,1,Not Specified
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0,0,No Loans
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0,1,Not Specified
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0,4,"Payday Loan, Student Loan, Mortgage Loan, and ..."
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0,3,"Home Equity Loan, Mortgage Loan, and Student Loan"
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude..."


*Note that multiple loan types have been clubbed into a single string of text. If I wish to look at each loan, I'd split the column by  the comma between the loan types and find the distribution of each resulting column. I could also establish a list of secured and unsecured loans, and find out how many of each a person has.*

 *However, for the sake of this analysis, I'll leave the column alone.*

 ### 13. Minimum Delay from due date
*If the minimum delay from the due date is 0, then the person has made at least one payment on time. If they've delayed payments by at least 1 day, they haven't made a single payment on time. While it's possible they're not doing well financially and cannot afford to repay the loan at the moment, some people may also have malicious intentions while taking the loan (or may think of it as free money). Someone in financial trouble would likely recover with time, and it would be possible to restructure a loan schedule for them (thus ensuring they get the time they need to recover, and the company doesn't lose money on that loan), but someone who has no intention of repaying would eventually cost too much to recover the money from, in order to make it worth the effort - sometimes, even more than what the company would've received as the principal, interest and late fees.*

*Thus, to identify those who I can recover money from (even if they've been unable to repay their current loans on time), and those I cannot, I'd need to look at the minimum and maximum delay from due date, among other features.*

In [40]:
# Grouping delay from due date by customer ID and finding the minimum
##
min_delay_from_due_date = train.groupby('Customer_ID')['Delay_from_due_date'].min()

# Converting the grouping into a dictionary
##
min_delay_from_due_date_dict = min_delay_from_due_date.to_dict()

# Mapping the dictionary onto the dataframe being created
##
df['Minimum Delay From Due Date'] = df['Customer ID'].map(min_delay_from_due_date_dict)

# Displaying the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards,Number of Loans per Month,Most Common Loan Type,Minimum Delay From Due Date
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0,1,Credit-Builder Loan,3
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0,3,"Auto Loan, Auto Loan, and Not Specified",5
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0,1,Not Specified,0
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0,0,No Loans,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0,1,Not Specified,9
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0,4,"Payday Loan, Student Loan, Mortgage Loan, and ...",1
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0,3,"Home Equity Loan, Mortgage Loan, and Student Loan",3
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33


In [41]:
# Descriptive Statistics for the column

df['Minimum Delay From Due Date'].describe()

count    12500.000000
mean        18.730080
std         14.888651
min         -5.000000
25%          8.000000
50%         15.000000
75%         26.000000
max         62.000000
Name: Minimum Delay From Due Date, dtype: float64

*Note the minimum value of ($-5$) - while it's possible for some people to have paid their dues before time (therefore resulting in a negative difference between the payment date and the actual due date), I'm more interested in whether they've made any delayed payments (and early payments definitely aren't delayed ones). Also, while the actual reason for the negative values can be different in the database, early payments are not an unreasonable expectation.*

In [ ]:
# Replace negative values with 0
df.loc[df['Minimum Delay From Due Date'] < 0, 'Minimum Delay From Due Date'] = 0

# Descriptive Statistics

df['Minimum Delay From Due Date'].describe()

count    12500.000000
mean        18.811600
std         14.777154
min          0.000000
25%          8.000000
50%         15.000000
75%         26.000000
max         62.000000
Name: Minimum Delay From Due Date, dtype: float64

*Note the fact that $25\%$ of the people in this dataset have been at least 26 - 62 days late on their payments (which is basically 1 - 2 months). While the internal credit policies of multiple banks or NBFCs may tolerate a certain minimum delinquency - with a higher threshold on NBFCs than banks - they often do not tolerate a minimum delinquency of over 2 months.*

### 14. Maximum Delay from due date

In [43]:
# Grouping delay from due date by customer ID and finding the maximum
##
max_delay_from_due_date = train.groupby('Customer_ID')['Delay_from_due_date'].max()

# Converting the grouping into a dictionary
##
max_delay_from_due_date = max_delay_from_due_date.to_dict()

# Mapping the dictionary onto the dataframe being created
##
df['Maximum Delay From Due Date'] = df['Customer ID'].map(max_delay_from_due_date)

# Displaying the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards,Number of Loans per Month,Most Common Loan Type,Minimum Delay From Due Date,Maximum Delay From Due Date
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",0,8
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0,1,Credit-Builder Loan,3,7
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0,3,"Auto Loan, Auto Loan, and Not Specified",5,13
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0,1,Not Specified,0,7
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0,0,No Loans,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0,1,Not Specified,9,14
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0,4,"Payday Loan, Student Loan, Mortgage Loan, and ...",1,4
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0,3,"Home Equity Loan, Mortgage Loan, and Student Loan",3,7
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,35


In [44]:
# Descriptive Statistics

df['Maximum Delay From Due Date'].describe()

count    12500.00000
mean        23.40848
std         14.87503
min          0.00000
25%         12.00000
50%         20.00000
75%         30.00000
max         67.00000
Name: Maximum Delay From Due Date, dtype: float64

*Note that $75\%$ of the population in this dataset have delayed their payments by a maximum of a month. Since all of these values seem perfectly respectable (a rarity in most real life data), I can move on without cleaning this column any further.*

### 15. Changed Credit Limit
*This refers to the percentage change in credit card limit. An increased credit limitcan be interpreted as a sign of trust from another financial institution.*

In [45]:
# Ensuring all values are numeric
##
train['Changed_Credit_Limit']= train['Changed_Credit_Limit'].replace({'_':0," ":0})
train['Changed_Credit_Limit'] = train['Changed_Credit_Limit'].astype(str).str.replace(r'[^0-9.]', '', regex=True).astype(float)

# Grouping credit card limit by customer ID and aggregating by mean
##
changed_credit_limit = train.groupby('Customer_ID')['Changed_Credit_Limit'].mean().round(2)

# Converting the grouping to a dictionary
##
changed_credit_limit_dict = changed_credit_limit.to_dict()

# Mapping the dictionary onto the dataset being created
##
df['Average % Change in Credit Limit'] = df['Customer ID'].map(changed_credit_limit_dict)

# Print the dataset
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards,Number of Loans per Month,Most Common Loan Type,Minimum Delay From Due Date,Maximum Delay From Due Date,Average % Change in Credit Limit
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",0,8,8.99
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0,1,Credit-Builder Loan,3,7,5.80
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0,3,"Auto Loan, Auto Loan, and Not Specified",5,13,7.85
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0,1,Not Specified,0,7,2.00
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0,0,No Loans,1,10,2.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0,1,Not Specified,9,14,5.32
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0,4,"Payday Loan, Student Loan, Mortgage Loan, and ...",1,4,5.82
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0,3,"Home Equity Loan, Mortgage Loan, and Student Loan",3,7,5.38
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,35,18.68


In [46]:
# Descriptive Stats

df['Average % Change in Credit Limit'].describe()

count    12500.000000
mean        10.246804
std          6.341622
min          0.460000
25%          5.377500
50%          9.160000
75%         14.300000
max         31.120000
Name: Average % Change in Credit Limit, dtype: float64

*The credit limit of the people in this dataset has changed between 0% and 31.12%, with an average change of 10.24% and standard deviation of 6.34% - all of which seem like respectable numbers.*

### 16. Credit Mix

In [47]:
# Ensuring the values of credit mix which aren't available in the data are labelled as such
##
train['Credit_Mix'] = train['Credit_Mix'].replace({'_': 'Not Available'})

# Grouping credit mix by customer ID and aggregating by mode
##
credit_mix = train.groupby('Customer_ID')['Credit_Mix'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])

# Converting the groupings into a dictionary
##
credit_mix_dict = credit_mix.to_dict()

# Mapping the dictionary onto the dataframe being created
##
df['Most Common Credit Mix'] = df['Customer ID'].map(credit_mix_dict)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards,Number of Loans per Month,Most Common Loan Type,Minimum Delay From Due Date,Maximum Delay From Due Date,Average % Change in Credit Limit,Most Common Credit Mix
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",0,8,8.99,Good
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0,1,Credit-Builder Loan,3,7,5.80,Good
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0,3,"Auto Loan, Auto Loan, and Not Specified",5,13,7.85,Good
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0,1,Not Specified,0,7,2.00,Good
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0,0,No Loans,1,10,2.58,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0,1,Not Specified,9,14,5.32,Good
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0,4,"Payday Loan, Student Loan, Mortgage Loan, and ...",1,4,5.82,Good
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0,3,"Home Equity Loan, Mortgage Loan, and Student Loan",3,7,5.38,Good
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,35,18.68,Bad


In [48]:
# The distribution of the column

df['Most Common Credit Mix'].value_counts()

Most Common Credit Mix
Standard         5395
Good             3767
Bad              2939
Not Available     399
Name: count, dtype: int64

*The credit mix of 399 people out of 12,500 is not in the dataset. This might be an error in either recording or retrieving data, but unless I have a database I can go back to, there's very little I can do about it.*

### 17. Outstanding Debt

In [49]:
# Converting the values in the column to numeric
##
train['Outstanding_Debt'] = train['Outstanding_Debt'].astype(str).str.replace(r'[^0-9.]', '', regex=True).astype(float)

# Grouping outstanding loans by Customer ID and aggregating by mean
outstanding_debt = train.groupby('Customer_ID')['Outstanding_Debt'].mean().round(2)

# Grouping credit mix by customer ID and aggregating by mode
##
outstanding_debt_dict = outstanding_debt.to_dict()

# Converting the groupings into a dictionary
##
df['Average Outstanding Debt'] = df['Customer ID'].map(outstanding_debt_dict)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards,Number of Loans per Month,Most Common Loan Type,Minimum Delay From Due Date,Maximum Delay From Due Date,Average % Change in Credit Limit,Most Common Credit Mix,Average Outstanding Debt
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",0,8,8.99,Good,809.98
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0,1,Credit-Builder Loan,3,7,5.80,Good,605.03
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0,3,"Auto Loan, Auto Loan, and Not Specified",5,13,7.85,Good,1303.01
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0,1,Not Specified,0,7,2.00,Good,632.46
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0,0,No Loans,1,10,2.58,Standard,943.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0,1,Not Specified,9,14,5.32,Good,1079.48
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0,4,"Payday Loan, Student Loan, Mortgage Loan, and ...",1,4,5.82,Good,897.16
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0,3,"Home Equity Loan, Mortgage Loan, and Student Loan",3,7,5.38,Good,620.64
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,35,18.68,Bad,3571.70


In [50]:
# Descriptive Statistics

df['Average Outstanding Debt'].describe()

count    12500.000000
mean      1426.220376
std       1155.169458
min          0.230000
25%        566.072500
50%       1166.155000
75%       1945.962500
max       4998.070000
Name: Average Outstanding Debt, dtype: float64

*The people in this dataset have between 0.23 and 4998.07 in outstanding debt, with an average of 1426.22 and a standard deviation of 1155.17 - again, perfectly respectable figures, despite the high standard deviation (and thereby, high variance - which again, isn't entirely unexpected in financial data)*

### 18. Number of Credit Enquiries

In [51]:
# Grouping outstanding loans by Customer ID and aggregating by mean
##
credit_inquiries = train.groupby('Customer_ID')['Num_Credit_Inquiries'].mean()

# Converting the groupings into a dictionary
##
credit_inquiries_dict = credit_inquiries.to_dict()

# Mapping the dictionary onto the dataframe being constructed
##
df['Average No. of Credit Inquiries'] = df['Customer ID'].map(credit_inquiries_dict).round(0).astype(int)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards,Number of Loans per Month,Most Common Loan Type,Minimum Delay From Due Date,Maximum Delay From Due Date,Average % Change in Credit Limit,Most Common Credit Mix,Average Outstanding Debt,Average No. of Credit Inquiries
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",0,8,8.99,Good,809.98,4
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0,1,Credit-Builder Loan,3,7,5.80,Good,605.03,2
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0,3,"Auto Loan, Auto Loan, and Not Specified",5,13,7.85,Good,1303.01,3
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0,1,Not Specified,0,7,2.00,Good,632.46,4
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0,0,No Loans,1,10,2.58,Standard,943.86,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0,1,Not Specified,9,14,5.32,Good,1079.48,1
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0,4,"Payday Loan, Student Loan, Mortgage Loan, and ...",1,4,5.82,Good,897.16,7
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0,3,"Home Equity Loan, Mortgage Loan, and Student Loan",3,7,5.38,Good,620.64,3
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,35,18.68,Bad,3571.70,9


In [ ]:
# Descriptive statistics
 
df['Average No. of Credit Inquiries'].describe()

count    12500.000000
mean        27.695120
std         68.833408
min          0.000000
25%          3.000000
50%          6.000000
75%         10.000000
max        602.000000
Name: Average No. of Credit Inquiries, dtype: float64

*Note that 25% of the people in this dataset have an average of 10 - 602 credit enquiries (the upper end of which is questinably high), and the standard deviation is approx. 2.5 times the mean (suggesting the variance is higher than expected). To deal with this, I'll resort to my old tactic of replacing the values over $Q_{3} + 1.5 \times IQR$ with the mode.*

In [53]:
#Defining the first quartile, Q1, the third quartile, Q3, and the inter-quartile range for 'Number of Credit Cards'
##

# First Quartile
Q1 = df['Average No. of Credit Inquiries'].quantile(0.25)

# Third Quartile
Q3 = df['Average No. of Credit Inquiries'].quantile(0.75)

# Inter - Quartile Range
IQR = Q3 - Q1


# Finding Q3 + 1.5*IQR
##
upper_bound = Q3 + 1.5 * IQR



# Grouping the number of credit cards in 'train' by customer ID, and aggregated by mode
##
mode_per_customer = train.groupby('Customer_ID')['Num_Credit_Inquiries'].agg(lambda x: x.mode()[0])


# Defining a function to replacing the number of cards for those who fall above the defined upper bound with the mode
##
def replace_outliers(row):
    if row['Average No. of Credit Inquiries'] > upper_bound:
        # Find the mode for this specific 'Customer_ID' from 'train'
        return mode_per_customer.get(row['Customer ID'], row['Average No. of Credit Inquiries'])
    return row['Average No. of Credit Inquiries']

# Applying the function to the the dataframe
##
df['Average No. of Credit Inquiries'] = df.apply(replace_outliers, axis=1)

# Display the updated DataFrame
df


,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards,Number of Loans per Month,Most Common Loan Type,Minimum Delay From Due Date,Maximum Delay From Due Date,Average % Change in Credit Limit,Most Common Credit Mix,Average Outstanding Debt,Average No. of Credit Inquiries
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",0,8,8.99,Good,809.98,4.0
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0,1,Credit-Builder Loan,3,7,5.80,Good,605.03,2.0
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0,3,"Auto Loan, Auto Loan, and Not Specified",5,13,7.85,Good,1303.01,3.0
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0,1,Not Specified,0,7,2.00,Good,632.46,4.0
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0,0,No Loans,1,10,2.58,Standard,943.86,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0,1,Not Specified,9,14,5.32,Good,1079.48,1.0
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0,4,"Payday Loan, Student Loan, Mortgage Loan, and ...",1,4,5.82,Good,897.16,7.0
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0,3,"Home Equity Loan, Mortgage Loan, and Student Loan",3,7,5.38,Good,620.64,3.0
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,35,18.68,Bad,3571.70,9.0


In [54]:
# Descriptive Statistics

df['Average No. of Credit Inquiries'].describe()

count    12500.000000
mean         5.821680
std          3.781786
min          0.000000
25%          3.000000
50%          5.000000
75%          8.000000
max         20.000000
Name: Average No. of Credit Inquiries, dtype: float64

*Now, the average number of credit enquiries lies between 0 and 20, with a mean of 5.82 (~ 6) enquiries and a standard deviation of 3.78 (~ 4) enquiries*

### 19. Credit Utilization Ratio
*The Credit utilization ratio is the percentage of the credit card limit that a person is using. A low ratio is an indicator that the person is in control of their finances (and doesn't need to depend on credit a lot)*

In [55]:
#Grouping Credit Utilization Ratio by customer ID and aggregating by mean
##
credit_utilization_ratio = train.groupby('Customer_ID')['Credit_Utilization_Ratio'].mean().round(2)

# Converting the groupings into a dictionary
##
credit_utilization_ratio_dict = credit_utilization_ratio.to_dict()

# Mapping the dictionary onto the dataframe being condtructed
##
df['Average Credit Utilization Ratio'] = df['Customer ID'].map(credit_utilization_ratio_dict)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,Interest Rate on Credit Cards,Number of Loans per Month,Most Common Loan Type,Minimum Delay From Due Date,Maximum Delay From Due Date,Average % Change in Credit Limit,Most Common Credit Mix,Average Outstanding Debt,Average No. of Credit Inquiries,Average Credit Utilization Ratio
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",0,8,8.99,Good,809.98,4.0,27.16
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,6.0,1,Credit-Builder Loan,3,7,5.80,Good,605.03,2.0,33.48
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,8.0,3,"Auto Loan, Auto Loan, and Not Specified",5,13,7.85,Good,1303.01,3.0,35.49
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,4.0,1,Not Specified,0,7,2.00,Good,632.46,4.0,31.78
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,5.0,0,No Loans,1,10,2.58,Standard,943.86,4.0,30.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,6.0,1,Not Specified,9,14,5.32,Good,1079.48,1.0,31.58
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,5.0,4,"Payday Loan, Student Loan, Mortgage Loan, and ...",1,4,5.82,Good,897.16,7.0,35.09
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,5.0,3,"Home Equity Loan, Mortgage Loan, and Student Loan",3,7,5.38,Good,620.64,3.0,34.60
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,29.0,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,35,18.68,Bad,3571.70,9.0,30.27


In [56]:
# Desctiptive statistics

df['Average Credit Utilization Ratio'].describe()

count    12500.000000
mean        32.285128
std          2.060554
min         25.480000
25%         30.850000
50%         32.240000
75%         33.600000
max         42.400000
Name: Average Credit Utilization Ratio, dtype: float64

*The people in this datasetuse around 25.5% - 42.4% of their credit limits, with an average of 32.3% and standard deviation of 2.06% - all of which are pretty respectable figures.*

### 20. Payment of minimum amount
*This refers to whether the person is making the minimum repayment required on their credit card to keep it in good standing.*

In [57]:
# Grouping payment status of minimum amount by customer ID and aggregating by mode
##
payment_status = train.groupby('Customer_ID')['Payment_of_Min_Amount'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])

# Converting the grouping into a dictionary
##
payment_status_dict = payment_status.to_dict()

# Mapping the dictionary onto the dataframe being constructed
##
df['Most Common Payment of Minimum Amount Status'] = df['Customer ID'].map(payment_status_dict)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,...,Number of Loans per Month,Most Common Loan Type,Minimum Delay From Due Date,Maximum Delay From Due Date,Average % Change in Credit Limit,Most Common Credit Mix,Average Outstanding Debt,Average No. of Credit Inquiries,Average Credit Utilization Ratio,Most Common Payment of Minimum Amount Status
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,...,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",0,8,8.99,Good,809.98,4.0,27.16,No
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,...,1,Credit-Builder Loan,3,7,5.80,Good,605.03,2.0,33.48,No
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,...,3,"Auto Loan, Auto Loan, and Not Specified",5,13,7.85,Good,1303.01,3.0,35.49,No
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,...,1,Not Specified,0,7,2.00,Good,632.46,4.0,31.78,No
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,...,0,No Loans,1,10,2.58,Standard,943.86,4.0,30.29,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,...,1,Not Specified,9,14,5.32,Good,1079.48,1.0,31.58,No
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,...,4,"Payday Loan, Student Loan, Mortgage Loan, and ...",1,4,5.82,Good,897.16,7.0,35.09,No
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,...,3,"Home Equity Loan, Mortgage Loan, and Student Loan",3,7,5.38,Good,620.64,3.0,34.60,No
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,...,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,35,18.68,Bad,3571.70,9.0,30.27,Yes


In [58]:
# Distribution of values

df['Most Common Payment of Minimum Amount Status'].value_counts()

Most Common Payment of Minimum Amount Status
Yes    7360
No     5031
NM      109
Name: count, dtype: int64

In [59]:
# Replacing 'NM' wit "Not Available"

df['Most Common Payment of Minimum Amount Status'] = df['Most Common Payment of Minimum Amount Status'].replace({'NM': "Not Available"})

df['Most Common Payment of Minimum Amount Status'].value_counts()

Most Common Payment of Minimum Amount Status
Yes              7360
No               5031
Not Available     109
Name: count, dtype: int64

### 21. Total EMI Amount per Month:
*The total EMI someone pays in a month is a good representation of their debt obligations. This is also used to calculate a measure called Debt Servicing Ratio (DSR), which tells me the percentage of a person's income that's going into repaying debt. A high DSR, regardless of other variables would dissuade me from giving the applicant a loan, as it's likely they would find it difficult to repay me on time.*

In [60]:
# Grouping Total EMI per month by Customer ID and aggregating by mean
##
emi = train.groupby('Customer_ID')['Total_EMI_per_month'].mean().round(2)

# Converting the groupings to a dictionary
##
emi_dict = emi.to_dict()

# Mapping the dictionary onto the dataframe being made
##
df['Average EMI per Month'] = df['Customer ID'].map(emi_dict)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,...,Most Common Loan Type,Minimum Delay From Due Date,Maximum Delay From Due Date,Average % Change in Credit Limit,Most Common Credit Mix,Average Outstanding Debt,Average No. of Credit Inquiries,Average Credit Utilization Ratio,Most Common Payment of Minimum Amount Status,Average EMI per Month
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,...,"Auto Loan, Credit-Builder Loan, Personal Loan,...",0,8,8.99,Good,809.98,4.0,27.16,No,49.57
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,...,Credit-Builder Loan,3,7,5.80,Good,605.03,2.0,33.48,No,18.82
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,...,"Auto Loan, Auto Loan, and Not Specified",5,13,7.85,Good,1303.01,3.0,35.49,No,246.99
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,...,Not Specified,0,7,2.00,Good,632.46,4.0,31.78,No,16.42
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,...,No Loans,1,10,2.58,Standard,943.86,4.0,30.29,Yes,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,...,Not Specified,9,14,5.32,Good,1079.48,1.0,31.58,No,6392.35
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,...,"Payday Loan, Student Loan, Mortgage Loan, and ...",1,4,5.82,Good,897.16,7.0,35.09,No,9412.35
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,...,"Home Equity Loan, Mortgage Loan, and Student Loan",3,7,5.38,Good,620.64,3.0,34.60,No,84.21
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,...,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,35,18.68,Bad,3571.70,9.0,30.27,Yes,1567.34


In [61]:
# Descriptive Statistics
df['Average EMI per Month'].describe()

count    12500.000000
mean      1403.118244
std       2914.621362
min          0.000000
25%         41.107500
50%        105.295000
75%        369.932500
max      29986.740000
Name: Average EMI per Month, dtype: float64

*Note that the standard deviation is greater than the mean. This implies a high variance within the column, with $25\%$ of the people in this dataset paying between 369.93 (~ 370) and 29,986.74 (~ 29,987) in EMI on average each month.*

### 22. Debt Servicing Ratio

In [62]:
# Debt Servicing Ratio = Total Monthly EMI / Salary; in this case average monthly EMI / Salary
##
df['Average Debt Servicing Ratio in Percentage'] = (df['Average EMI per Month'] / df['Average Monthly Income'])*100

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,...,Minimum Delay From Due Date,Maximum Delay From Due Date,Average % Change in Credit Limit,Most Common Credit Mix,Average Outstanding Debt,Average No. of Credit Inquiries,Average Credit Utilization Ratio,Most Common Payment of Minimum Amount Status,Average EMI per Month,Average Debt Servicing Ratio in Percentage
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,...,0,8,8.99,Good,809.98,4.0,27.16,No,49.57,3.112051
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,...,3,7,5.80,Good,605.03,2.0,33.48,No,18.82,0.648074
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,...,5,13,7.85,Good,1303.01,3.0,35.49,No,246.99,2.070289
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,...,0,7,2.00,Good,632.46,4.0,31.78,No,16.42,0.642036
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,...,1,10,2.58,Standard,943.86,4.0,30.29,Yes,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,...,9,14,5.32,Good,1079.48,1.0,31.58,No,6392.35,178.790989
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,...,1,4,5.82,Good,897.16,7.0,35.09,No,9412.35,677.132868
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,...,3,7,5.38,Good,620.64,3.0,34.60,No,84.21,2.717319
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,...,33,35,18.68,Bad,3571.70,9.0,30.27,Yes,1567.34,94.026672


In [63]:
# Descriptive Statistics

df['Average Debt Servicing Ratio in Percentage'].describe()

count    12500.000000
mean        58.135214
std        165.295476
min          0.000000
25%          1.606223
50%          3.315832
75%          7.376545
max       1902.027675
Name: Average Debt Servicing Ratio in Percentage, dtype: float64

*Note the maximum value of $1,902.2\%$, with $Q_{3}$ being $7.37\%$ - showing extreme outliers. While people do pay twice or thrice their income in EMIs (which might suggest they've fallen into a debt trap and are taking on more debt to repay previous loans), $1902\%$ of their income is framnky impossible to acheive, since even a DSR of twice or thrice a person's income raises red flags with lenders. These extremities may just be outliers (if the values are correct by whatever twist of data), or a glitch in recording, retrieving or handling data. To deal with this in the current data, I'll replace values over $Q_{3} + 1.5 \times IQR$ with the median of the column (since it's a calculated column and not derived from the source data).*

In [64]:
#Defining the first quartile, Q1, the third quartile, Q3, and the inter-quartile range for 'Number of Credit Cards'
##

# First Quartile
Q1 = df['Average Debt Servicing Ratio in Percentage'].quantile(0.25)

# Third Quartile
Q3 = df['Average Debt Servicing Ratio in Percentage'].quantile(0.75)

# Inter - Quartile Range
IQR = Q3 - Q1


# Finding Q3 + 1.5*IQR
##
upper_bound = Q3 + 1.5 * IQR



# Calculating the median of the column 'Average Debt Servicing Ratio in Percentage'
median_value = df['Average Debt Servicing Ratio in Percentage'].median()

# Replacing values over the upper bound with the median value
df['Average Debt Servicing Ratio in Percentage'] = df['Average Debt Servicing Ratio in Percentage'].apply(
    lambda x: median_value if x > upper_bound else x
)

# Display descriptive Statistics
df['Average Debt Servicing Ratio in Percentage'].describe()

count    12500.000000
mean         3.015587
std          2.085763
min          0.000000
25%          1.606223
50%          3.315776
75%          3.646874
max         15.848589
Name: Average Debt Servicing Ratio in Percentage, dtype: float64

*Now note that the maximum value has falen to $15.84\%$ (~ $16\%$), and the mean and standard deviation are approx. $3\%$ and $2\%$ respectively*

### 23. Average Monthly Investments
*The amount of money someone can invest in a month is indicative of their disposable income (technically, income left after taxes, but often, income left after all necessary expenses have been met). Someone with more disposable income would find it easier to repay loans.*

In [65]:
# Retaining only the numeric parts of the source column
##
train['Amount_invested_monthly'] = train['Amount_invested_monthly'].astype(str).str.replace(r'[^0-9.]', '', regex=True)

# Replace empty strings with NaN before converting to float
##
train['Amount_invested_monthly'] = train['Amount_invested_monthly'].replace('', np.nan).astype(float)

# Grouping Amount_invested_monthly by customer ID and aggregating by mean
##
investment = train.groupby('Customer_ID')['Amount_invested_monthly'].mean().round(2)

# Converting the groupings into a dictionary
##
investment_dict = investment.to_dict()

# Mapping the dictionary onto the dataframe being made
##
df['Average Amount Invested Monthly'] = df['Customer ID'].map(investment_dict)

# Display the dataset
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,...,Maximum Delay From Due Date,Average % Change in Credit Limit,Most Common Credit Mix,Average Outstanding Debt,Average No. of Credit Inquiries,Average Credit Utilization Ratio,Most Common Payment of Minimum Amount Status,Average EMI per Month,Average Debt Servicing Ratio in Percentage,Average Amount Invested Monthly
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,...,8,8.99,Good,809.98,4.0,27.16,No,49.57,3.112051,98.35
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,...,7,5.80,Good,605.03,2.0,33.48,No,18.82,0.648074,95.64
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,...,13,7.85,Good,1303.01,3.0,35.49,No,246.99,2.070289,2772.30
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,...,7,2.00,Good,632.46,4.0,31.78,No,16.42,0.642036,1355.46
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,...,10,2.58,Standard,943.86,4.0,30.29,Yes,0.00,0.000000,135.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,...,14,5.32,Good,1079.48,1.0,31.58,No,6392.35,3.315832,174.09
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,...,4,5.82,Good,897.16,7.0,35.09,No,9412.35,3.315832,1509.78
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,...,7,5.38,Good,620.64,3.0,34.60,No,84.21,2.717319,123.47
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,...,35,18.68,Bad,3571.70,9.0,30.27,Yes,1567.34,3.315832,100.99


In [66]:
# Display descriptive stats

df['Average Amount Invested Monthly'].describe()

count    12500.000000
mean       637.569720
std        751.542773
min         15.290000
25%        117.185000
50%        233.590000
75%       1336.540000
max       5748.510000
Name: Average Amount Invested Monthly, dtype: float64

*While the standard deviation is higher than the mean (indicating a higher than desirable variance), the numbers by themselves do not seem out of the expected. So, I'll leave the column alone.*

### 24. Payment Behaviour
*Payment behaviour is particularly helpful when trying to detect fraud. For example, if the bank notices a single spending of ₹ 10,000 from the savings account of someone who spends around ₹ 100 - ₹ 150 a day, that transaction raises a red flag and the bank would confirm with the person whether they made the transaction or not. However, if a merchant regularly makes transactions of ₹ 7,000 - ₹ 12,000 from their current account multiple times in a day, then a single transaction of ₹ 10,000 doesn't raise a red flag, though a single transaction of ₹ 2,00,000 probably would.*

In [ ]:
# Finding the unique payment behaviours possible
##
train['Payment_Behaviour'].unique()

array(['High_spent_Small_value_payments',
       'Low_spent_Large_value_payments',
       'Low_spent_Medium_value_payments',
       'Low_spent_Small_value_payments',
       'High_spent_Medium_value_payments', '!@9#%8',
       'High_spent_Large_value_payments'], dtype=object)

*Notice the string of characters, '!@9#%8', among the values of Payment Behaviour. While I'd typically go back to the db to check, for this dataset, I'll replace the string with 'Not Available'*

In [68]:
# Replacing the string of characters with 'Not Available'
##
train['Payment_Behaviour'] = train['Payment_Behaviour'].replace({'!@9#%8': 'Not Available'})

# Grouping payment_behaviour by Customer ID and aggregating by Customer ID
##
payment_behavior = train.groupby('Customer_ID')['Payment_Behaviour'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x)

# Converting the groupings into a dictionary
##
payment_behavior_dict = payment_behavior.to_dict()

# Mapping the dictionary onto the dataframe being constructed
##
df['Most Common Payment Behavior'] = df['Customer ID'].map(payment_behavior_dict)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,...,Average % Change in Credit Limit,Most Common Credit Mix,Average Outstanding Debt,Average No. of Credit Inquiries,Average Credit Utilization Ratio,Most Common Payment of Minimum Amount Status,Average EMI per Month,Average Debt Servicing Ratio in Percentage,Average Amount Invested Monthly,Most Common Payment Behavior
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,...,8.99,Good,809.98,4.0,27.16,No,49.57,3.112051,98.35,High_spent_Medium_value_payments
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,...,5.80,Good,605.03,2.0,33.48,No,18.82,0.648074,95.64,High_spent_Large_value_payments
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,...,7.85,Good,1303.01,3.0,35.49,No,246.99,2.070289,2772.30,High_spent_Small_value_payments
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,...,2.00,Good,632.46,4.0,31.78,No,16.42,0.642036,1355.46,Low_spent_Large_value_payments
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,...,2.58,Standard,943.86,4.0,30.29,Yes,0.00,0.000000,135.48,High_spent_Medium_value_payments
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,...,5.32,Good,1079.48,1.0,31.58,No,6392.35,3.315832,174.09,High_spent_Medium_value_payments
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,...,5.82,Good,897.16,7.0,35.09,No,9412.35,3.315832,1509.78,Low_spent_Small_value_payments
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,...,5.38,Good,620.64,3.0,34.60,No,84.21,2.717319,123.47,Low_spent_Small_value_payments
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,...,18.68,Bad,3571.70,9.0,30.27,Yes,1567.34,3.315832,100.99,High_spent_Large_value_payments


In [69]:
# Display the distribution of values in the column
##
df['Most Common Payment Behavior'].value_counts()

Most Common Payment Behavior
Low_spent_Small_value_payments      3833
High_spent_Medium_value_payments    3048
High_spent_Large_value_payments     2675
Low_spent_Medium_value_payments     1111
High_spent_Small_value_payments      960
Low_spent_Large_value_payments       704
Not Available                        169
Name: count, dtype: int64

### 25. Monthly Balance

In [70]:
# Extracting the numeric parts of 'Monthly Balance' in the source data
##
train['Monthly_Balance'] = train['Monthly_Balance'].astype(str).str.replace(r'[^0-9.]', '', regex=True)

# Replacing empty spaces with null values before converting to float
##
train['Monthly_Balance'] = train['Monthly_Balance'].replace('', np.nan).astype(float)

# Grouping the values by Customer ID and aggregating by mean
##
balance = train.groupby('Customer_ID')['Monthly_Balance'].mean().round(2)

# Converting the groupings into a dictionary
##
balance_dict = balance.to_dict()

# Mapping the dictionary onto the dataframe being constructed
##
df['Average Monthly Balance'] = df['Customer ID'].map(balance_dict)

# Display the dataframe
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,...,Most Common Credit Mix,Average Outstanding Debt,Average No. of Credit Inquiries,Average Credit Utilization Ratio,Most Common Payment of Minimum Amount Status,Average EMI per Month,Average Debt Servicing Ratio in Percentage,Average Amount Invested Monthly,Most Common Payment Behavior,Average Monthly Balance
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,19114.12,1592.84,22,3,4,...,Good,809.98,4.0,27.16,No,49.57,3.112051,98.35,High_spent_Medium_value_payments,304.56
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,34847.84,2903.99,26,2,4,...,Good,605.03,2.0,33.48,No,18.82,0.648074,95.64,High_spent_Large_value_payments,454.34
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,143162.64,11930.22,17,1,5,...,Good,1303.01,3.0,35.49,No,246.99,2.070289,2772.30,High_spent_Small_value_payments,852.90
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,30689.89,2557.49,17,2,5,...,Good,632.46,4.0,31.78,No,16.42,0.642036,1355.46,Low_spent_Large_value_payments,389.94
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,35547.71,2962.31,30,7,5,...,Standard,943.86,4.0,30.29,Yes,0.00,0.000000,135.48,High_spent_Medium_value_payments,414.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,42903.79,3575.32,28,0,4,...,Good,1079.48,1.0,31.58,No,6392.35,3.315832,174.09,High_spent_Medium_value_payments,412.35
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,16680.35,1390.03,0,1,1,...,Good,897.16,7.0,35.09,No,9412.35,3.315832,1509.78,Low_spent_Small_value_payments,288.21
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,37188.10,3099.01,29,1,4,...,Good,620.64,3.0,34.60,No,84.21,2.717319,123.47,Low_spent_Small_value_payments,372.02
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,20002.88,1666.91,5,10,8,...,Bad,3571.70,9.0,30.27,Yes,1567.34,3.315832,100.99,High_spent_Large_value_payments,294.78


In [71]:
# Suppress scientific notation
pd.options.display.float_format = '{:,.2f}'.format

# Display the descriptive statistics
df['Average Monthly Balance'].describe()

count                               12,500.00
mean        30,000,000,000,000,004,194,304.00
std      1,117,676,131,978,704,704,241,664.00
min                                     92.84
25%                                    284.58
50%                                    338.87
75%                                    465.88
max     41,666,666,666,666,664,357,003,264.00
Name: Average Monthly Balance, dtype: float64

*Note that one huge maximum value when $75\%$ of the people have less than 500 in their bank accounts. To deal, I'll replace the outliers on the higher end (i.e. $Q_{3} + 1.5 \times IQR$) with the median for that particular person.*

In [72]:
#Defining the first quartile, Q1, the third quartile, Q3, and the inter-quartile range for 'Number of Credit Cards'
##

# First Quartile
Q1 = df['Average Monthly Balance'].quantile(0.25)

# Third Quartile
Q3 = df['Average Monthly Balance'].quantile(0.75)

# Inter - Quartile Range
IQR = Q3 - Q1


# Finding Q3 + 1.5*IQR
##
upper_bound = Q3 + 1.5 * IQR



# Grouping the number of credit cards in 'train' by customer ID, and aggregated by mode
##
mode_per_customer = train.groupby('Customer_ID')['Monthly_Balance'].median()


# Defining a function to replacing the number of cards for those who fall above the defined upper bound with the mode
##
def replace_outliers(row):
    if row['Average Monthly Balance'] > upper_bound:
        # Find the mode for this specific 'Customer_ID' from 'train'
        return mode_per_customer.get(row['Customer ID'], row['Average Monthly Balance'])
    return row['Average Monthly Balance']

# Applying the function to the the dataframe
##
df['Average Monthly Balance'] = df.apply(replace_outliers, axis=1)

# Display the updated DataFrame
df


,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,...,Most Common Credit Mix,Average Outstanding Debt,Average No. of Credit Inquiries,Average Credit Utilization Ratio,Most Common Payment of Minimum Amount Status,Average EMI per Month,Average Debt Servicing Ratio in Percentage,Average Amount Invested Monthly,Most Common Payment Behavior,Average Monthly Balance
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,"19,114.12","1,592.84",22,3,4,...,Good,809.98,4.00,27.16,No,49.57,3.11,98.35,High_spent_Medium_value_payments,304.56
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,"34,847.84","2,903.99",26,2,4,...,Good,605.03,2.00,33.48,No,18.82,0.65,95.64,High_spent_Large_value_payments,454.34
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,"143,162.64","11,930.22",17,1,5,...,Good,"1,303.01",3.00,35.49,No,246.99,2.07,"2,772.30",High_spent_Small_value_payments,929.71
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,"30,689.89","2,557.49",17,2,5,...,Good,632.46,4.00,31.78,No,16.42,0.64,"1,355.46",Low_spent_Large_value_payments,389.94
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,"35,547.71","2,962.31",30,7,5,...,Standard,943.86,4.00,30.29,Yes,0.00,0.00,135.48,High_spent_Medium_value_payments,414.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,"42,903.79","3,575.32",28,0,4,...,Good,"1,079.48",1.00,31.58,No,"6,392.35",3.32,174.09,High_spent_Medium_value_payments,412.35
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,"16,680.35","1,390.03",0,1,1,...,Good,897.16,7.00,35.09,No,"9,412.35",3.32,"1,509.78",Low_spent_Small_value_payments,288.21
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,"37,188.10","3,099.01",29,1,4,...,Good,620.64,3.00,34.60,No,84.21,2.72,123.47,Low_spent_Small_value_payments,372.02
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,"20,002.88","1,666.91",5,10,8,...,Bad,"3,571.70",9.00,30.27,Yes,"1,567.34",3.32,100.99,High_spent_Large_value_payments,294.78


In [73]:
# Descriptive statistics

df['Average Monthly Balance'].describe()

count   12,500.00
mean       407.55
std        188.54
min         92.84
25%        284.58
50%        338.66
75%        465.38
max      1,427.80
Name: Average Monthly Balance, dtype: float64

*While it's entirely possible for the super rich to have super high bank balances, note that replacing those super high means with the medians for the same person has brought the maximum down to approx. 1,428. This is more indicative of aberrations in the data than of someone being super rich, since the median bank balance for these folks would also have been much higher. However, if the bank balance for a person really does spike by that much in a month, it would raise a few red flags about the source of the money.*

### 26. Credit Score


In [74]:
# Grouping credit score by customer ID and aggregating by mode. 
# The mode will give me what someone's credit score has been most of the time (or at least, what their most common position is)
##
credit_score = train.groupby('Customer_ID')['Credit_Score'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])

# Converting the grouping into a dictionary
##
credit_score_dict = credit_score.to_dict()

# Mapping the dictionary into the dataframe being constructed
##
df['Credit Score'] = df['Customer ID'].map(credit_score_dict)

# Display the dataset:
df

,Customer ID,Name,Age Group,Occupation,Count of Months,Annual Income,Average Monthly Income,Vintage in Years,Number of Bank Accounts,Number of Credit Cards,...,Average Outstanding Debt,Average No. of Credit Inquiries,Average Credit Utilization Ratio,Most Common Payment of Minimum Amount Status,Average EMI per Month,Average Debt Servicing Ratio in Percentage,Average Amount Invested Monthly,Most Common Payment Behavior,Average Monthly Balance,Credit Score
0,CUS_0xd40,Aaron Maashoh,A- 18 - 24,Scientist,8,"19,114.12","1,592.84",22,3,4,...,809.98,4.00,27.16,No,49.57,3.11,98.35,High_spent_Medium_value_payments,304.56,Good
1,CUS_0x21b1,Rick Rothackerj,B- 25 - 34,Not Available,8,"34,847.84","2,903.99",26,2,4,...,605.03,2.00,33.48,No,18.82,0.65,95.64,High_spent_Large_value_payments,454.34,Good
2,CUS_0x2dbc,Langep,B- 25 - 34,Not Available,8,"143,162.64","11,930.22",17,1,5,...,"1,303.01",3.00,35.49,No,246.99,2.07,"2,772.30",High_spent_Small_value_payments,929.71,Good
3,CUS_0xb891,Jasond,D- 45 - 59,Entrepreneur,8,"30,689.89","2,557.49",17,2,5,...,632.46,4.00,31.78,No,16.42,0.64,"1,355.46",Low_spent_Large_value_payments,389.94,Standard
4,CUS_0x1cdb,Deepaa,A- 18 - 24,Developer,8,"35,547.71","2,962.31",30,7,5,...,943.86,4.00,30.29,Yes,0.00,0.00,135.48,High_spent_Medium_value_payments,414.86,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0x372c,Lucia Mutikanik,A- 18 - 24,Lawyer,8,"42,903.79","3,575.32",28,0,4,...,"1,079.48",1.00,31.58,No,"6,392.35",3.32,174.09,High_spent_Medium_value_payments,412.35,Good
12496,CUS_0xf16,Maria Sheahanb,D- 45 - 59,Media Manager,8,"16,680.35","1,390.03",0,1,1,...,897.16,7.00,35.09,No,"9,412.35",3.32,"1,509.78",Low_spent_Small_value_payments,288.21,Good
12497,CUS_0xaf61,Chris Wickhamm,D- 45 - 59,Writer,8,"37,188.10","3,099.01",29,1,4,...,620.64,3.00,34.60,No,84.21,2.72,123.47,Low_spent_Small_value_payments,372.02,Good
12498,CUS_0x8600,Sarah McBridec,B- 25 - 34,Architect,8,"20,002.88","1,666.91",5,10,8,...,"3,571.70",9.00,30.27,Yes,"1,567.34",3.32,100.99,High_spent_Large_value_payments,294.78,Standard


*This brings us to the end of the exercise for 'train'. I have now created a dataframe that contains  25 columns about information about 12,500 people (excluding their names and customer IDs of course), which I can now use to answer the question, 'Who applied for a loan at my company and what do I know about them?'. If I replace 'train' with 'test' in this code (and make the necessary adjustments), I'll get a similar dataframe from 'test', which would then go into answering the same question, perhaps with slightly better insights. Now, I'll just get this data out into a csv file for further use.*

In [75]:
# Export df into a csv file

df.to_csv('Train_final.csv', index=False)